In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, timezone


In [8]:
df_orig = pd.read_csv('./experiment_data.csv')
print(df_orig.head())

   Unnamed: 0      noisy              original_time
0           0  65.621621  2016-05-21 11:59:32+00:00
1           1  62.380593  2016-05-22 11:59:32+00:00
2           2  65.974625  2016-05-23 11:59:32+00:00
3           3  65.964745  2016-05-24 11:59:32+00:00
4           4  65.706041  2016-05-25 11:59:32+00:00


    def detect_anomalies(self, new_df):
        time_steps = self.time_steps
        test_data = new_df.copy()
        test_data['noisy_scl'] = self.scaler.transform(test_data[['noisy']])
        X_test, y_test = self.create_dataset(test_data[['noisy_scl']], test_data.noisy_scl, time_steps)
        X_test_pred = self.model.predict(X_test)
        test_mae_loss = np.mean(np.abs(X_test_pred - X_test), axis=1)

        THRESHOLD = 0.48

        test_score_df = pd.DataFrame(test_data[time_steps:])
        test_score_df['loss'] = test_mae_loss
        test_score_df['threshold'] = THRESHOLD
        test_score_df['anomaly'] = test_score_df.loss > test_score_df.threshold
        test_score_df['noisy'] = test_data[time_steps:].noisy
        print('anomaly tested')
        print(test_score_df)
        return test_score_df

this anomaly detection section of my code, doesn't detect anomaly correctly, because we are giving it an interval in the data stream. We need to give whole one year. but it seems not possible.

We have self.training_df I want this: create a new dataframe, and in that dataframe replace the time interval with self.training_df.

In [15]:
df_orig.head()

,Unnamed: 0,noisy,original_time
0,0,65.621621,2016-05-21 11:59:32+00:00
1,1,62.380593,2016-05-22 11:59:32+00:00
2,2,65.974625,2016-05-23 11:59:32+00:00
3,3,65.964745,2016-05-24 11:59:32+00:00
4,4,65.706041,2016-05-25 11:59:32+00:00


In [18]:
# df_orig.set_index('original_time', inplace=True)
df = df_orig.copy()
df.index = pd.to_datetime(df.index)

start_date = pd.to_datetime(df.index.min())
end_date = start_date + timedelta(days=365)

train_data = df[(df.index >= start_date) & (df.index < end_date)]

start_date= end_date
end_date = start_date+ timedelta(days=365)
test_data =  = df[(df.index >= start_date) & (df.index < end_date)]

In [19]:
test_data.head()

,Unnamed: 0,noisy,original_time


In [10]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_data.original_time, y=train_data.noisy,
                    mode='lines',
                    name='rate'))
fig.update_layout(showlegend=True)
fig.show()

# Train

In [11]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed

scaler = StandardScaler()
scaler = scaler.fit(train_data[['noisy']])
train_data['noisy_scl'] = scaler.transform(train_data[['noisy']])
train_data.head()

def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 15
X_train, y_train = create_dataset(train_data[['noisy_scl']], train_data.noisy_scl, time_steps)

num_features=1
model = Sequential([
    LSTM(128, input_shape=(time_steps, num_features)),
    Dropout(0.2),
    RepeatVector(time_steps),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    TimeDistributed(Dense(num_features))                 
])

model.compile(loss='mae', optimizer='adam')
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    shuffle=False,
)

Epoch 1/50


/home/fatih.durmaz/cobblestone/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



65/65 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.4150 - val_loss: nan
Epoch 2/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.2973 - val_loss: nan
Epoch 3/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.2527 - val_loss: nan
Epoch 4/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.2353 - val_loss: nan
Epoch 5/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.2413 - val_loss: nan
Epoch 6/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.2132 - val_loss: nan
Epoch 7/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.2256 - val_loss: nan
Epoch 8/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.2155 - val_loss: nan
Epoch 9/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.2153 - val_loss: nan
Epoch 10/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.2098 - val_loss: nan
Epoch 11/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.2070 - val_loss: nan
Epoch 12/50
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.2109 - val_loss: nan
Epoch 13/50
65/65 ━━━━━━

# Test

In [14]:
test_data.head()

,Unnamed: 0,noisy,original_time


In [13]:


test_data['noisy_scl'] = scaler.transform(test_data[['noisy']])
X_test, y_test = create_dataset(test_data[['noisy_scl']], test_data.noisy_scl, time_steps)
X_test_pred = model.predict(X_test)
test_mae_loss = np.mean(np.abs(X_test_pred - X_test), axis=1)

THRESHOLD = 0.48

test_score_df = pd.DataFrame(test_data[time_steps:])
test_score_df['loss'] = test_mae_loss
test_score_df['threshold'] = THRESHOLD
test_score_df['anomaly'] = test_score_df.loss > test_score_df.threshold
test_score_df['noisy'] = test_data[time_steps:].noisy


ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by StandardScaler.

In [ ]:
test_score_df.head()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_data[time_steps:].index, y=test_score_df.loss,
                    mode='lines',
                    name='Test Loss'))
fig.add_trace(go.Scatter(x=test_data[time_steps:].index, y=test_score_df.threshold,
                    mode='lines',
                    name='Threshold'))
fig.update_layout(showlegend=True)
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=test_data.index, y=test_data.noisy,
                    mode='lines',
                    name='rate'))
fig.update_layout(showlegend=True)
fig.show()

In [ ]:
import panel as pn
import pandas as pd
import hvplot.pandas
"jupyter_bokeh"
"panel"
"hvplot"
# Create some example data
df = pd.DataFrame({
    'x': range(100),
    'y': range(100)
})

# Create a plot
plot = df.hvplot.line('x', 'y')

# Create a widget
slider = pn.widgets.IntSlider(name='Frequency', start=1, end=10, step=1, value=1)

# Define a function to update the plot based on the widget
@pn.depends(frequency=slider.param.value)
def update_plot(frequency):
    df['y'] = df['x'] * frequency
    return df.hvplot.line('x', 'y')

# Create a Panel layout
dashboard = pn.Column(
    '### Simple Panel Dashboard',
    slider,
    update_plot
)

# Serve the panel dashboard
dashboard.servable()
